SIGN LANGUAGE PREDICTION

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, TimeDistributed, Conv3D, MaxPooling3D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
import os
import tkinter as tk
from tkinter import filedialog
from tkinter import Label, Button
from PIL import Image, ImageTk

ASL Alphabet Model for Image Prediction

In [2]:
# Load and preprocess the ASL alphabet dataset
def load_asl_alphabet_dataset():
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    
    train_generator = datagen.flow_from_directory(
        'ASL alphabet',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        subset='training')

    validation_generator = datagen.flow_from_directory(
        'ASL alphabet',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        subset='validation')
    
    return train_generator, validation_generator

train_gen, val_gen = load_asl_alphabet_dataset()

Found 1392 images belonging to 30 classes.
Found 348 images belonging to 30 classes.


In [3]:
# Define the image prediction model
image_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_gen.num_classes, activation='softmax')
])

image_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\SARATHLAL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# Train the model
image_model.fit(train_gen, validation_data=val_gen, epochs=10)

Epoch 1/10


C:\Users\SARATHLAL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


44/44 ━━━━━━━━━━━━━━━━━━━━ 13s 232ms/step - accuracy: 0.4527 - loss: 2.6818 - val_accuracy: 0.5000 - val_loss: 2.4624
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.4999 - loss: 2.4526 - val_accuracy: 0.5000 - val_loss: 2.5577
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.5028 - loss: 2.3579 - val_accuracy: 0.4282 - val_loss: 2.5910
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.5003 - loss: 2.3168 - val_accuracy: 0.3333 - val_loss: 2.7041
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.5220 - loss: 2.1380 - val_accuracy: 0.3420 - val_loss: 2.6532
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.5234 - loss: 2.0122 - val_accuracy: 0.2787 - val_loss: 2.8885
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.5327 - loss: 1.8930 - val_accuracy: 0.2385 - val_loss: 3.0028
Epoch 8/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 0.5201 - loss: 1.7289 - val_accuracy: 0.2098 - val_loss: 

WLASL Video Model for Phrase Prediction

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, LSTM, TimeDistributed, Dropout
import json
import cv2
import numpy as np
import os

In [6]:
# Paths
base_dir = 'wlasl-videos/videos'
metadata_path = 'wlasl-videos/WLASL_v0.3.json'
class_list_path = 'wlasl-videos/wlasl_class_list.txt'

In [7]:
# Load metadata
with open(metadata_path, 'r') as f:
    wlasl_metadata = json.load(f)

In [8]:
# Load class list
with open(class_list_path, 'r') as f:
    class_list = f.read().splitlines()

In [9]:
# Track missing labels
missing_labels = []

# Extract video paths and labels
video_paths = []
labels = []

for entry in wlasl_metadata:
    gloss = entry['gloss']
    if gloss not in class_list:
        missing_labels.append(gloss)
        class_list.append(gloss)  # Add missing labels to the class list
    for instance in entry['instances']:
        video_path = os.path.join(base_dir, instance['video_id'] + '.mp4')
        if os.path.exists(video_path):
            video_paths.append(video_path)
            labels.append(class_list.index(gloss))
        else:
            print(f"Video file not found: {video_path}")

# Save the updated class list if needed
with open('wlasl-videos/updated_wlasl_class_list.txt', 'w') as f:
    for item in class_list:
        f.write(f"{item}\n")

print(f"Missing labels added to the class list: {missing_labels}")

Video file not found: wlasl-videos/videos\69241.mp4
Video file not found: wlasl-videos/videos\68011.mp4
Video file not found: wlasl-videos/videos\68012.mp4
Video file not found: wlasl-videos/videos\07068.mp4
Video file not found: wlasl-videos/videos\69302.mp4
Video file not found: wlasl-videos/videos\68538.mp4
Video file not found: wlasl-videos/videos\68042.mp4
Video file not found: wlasl-videos/videos\68660.mp4
Video file not found: wlasl-videos/videos\68041.mp4
Video file not found: wlasl-videos/videos\67594.mp4
Video file not found: wlasl-videos/videos\12306.mp4
Video file not found: wlasl-videos/videos\68028.mp4
Video file not found: wlasl-videos/videos\69054.mp4
Video file not found: wlasl-videos/videos\12328.mp4
Video file not found: wlasl-videos/videos\12329.mp4
Video file not found: wlasl-videos/videos\12330.mp4
Video file not found: wlasl-videos/videos\12312.mp4
Video file not found: wlasl-videos/videos\12331.mp4
Video file not found: wlasl-videos/videos\12332.mp4
Video file n

Video Preprocessing

In [ ]:
import numpy as np

def preprocess_video(video_path, img_size=(64, 64), max_frames=30):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened() and len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, img_size)
        frames.append(frame)
    cap.release()
    
    # Pad or trim the frames to max_frames
    if len(frames) < max_frames:
        for _ in range(max_frames - len(frames)):
            frames.append(np.zeros_like(frames[0]))  # Pad with black frames
    else:
        frames = frames[:max_frames]  # Trim to max_frames
    
    frames = np.array(frames) / 255.0
    return frames

# Preprocess all videos
X_data = []
y_data = []

for video_path, label in zip(video_paths, labels):
    frames = preprocess_video(video_path)
    X_data.append(frames)
    y_data.append(label)

# Convert to numpy arrays
X_data = np.array(X_data)
y_data = np.array(y_data)
